In [1]:
# Подготовка по хакатону

In [2]:
! pip install pymorphy2

In [3]:
! pip install nltk

In [4]:
! pip install -U gensim

In [6]:
import pandas as pd
import pymorphy2
import nltk
import re

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from gensim.models import *
from gensim import corpora

from google.colab import drive

In [10]:
# dict_themes = {
#   'Психология': ['психология', 'когнитивный', 'мышление', 'установка', 'ограничение', 'развитие', 'мотивация', 'человек'],
#   'Программирование': ['программа', 'программирование', 'приложение']
# }

# dict_themes['Психология']

dir_hackathon = '.'
dir_data_in = f'{dir_hackathon}/Data/In'
dir_data_out = f'{dir_hackathon}/Data/Out'

In [8]:
drive.mount('/content/gdrive')

NameError: name 'drive' is not defined

In [14]:
all_data_in = pd.read_excel(f'{dir_data_in}/dataset.xlsx', header=0)
all_data_in.columns = ['Номер продукции', 'Коды ТН ВЭД ЕАЭС', 'Технические регламенты', 'Группа продукции', 'Общее наименование продукции', 
                       'ИЛ', 'Заявитель', 'Адрес Заявителя', 'Изготовитель', 'Страна', 'Адрес изготовителя']
all_data_in.head(2)

,Номер продукции,Коды ТН ВЭД ЕАЭС,Технические регламенты,Группа продукции,Общее наименование продукции,ИЛ,Заявитель,Адрес Заявителя,Изготовитель,Страна,Адрес изготовителя
0,37632 Продукция,4016999708,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...,ИЛ Мопалия Миддендорфа,ООО Европейский гологлаз 6,РОССИЯ 461220 БАРАБАНОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Ара Леара 11,РОССИЯ,РОССИЯ 182375 ЛУКИНО ПСКОВСКАЯ ОБЛАСТЬ
1,37565 Продукция,7308909809,ТР ЕАЭС 043/2017 О требованиях к средствам обе...,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...,ИЛ Мопалия Миддендорфа,ООО Дроздовидная6,РОССИЯ 461182 ШЕСТАКОВКА ОРЕНБУРГСКАЯ ОБЛАСТЬ,ООО Аннамский палочник 11,РОССИЯ,РОССИЯ 182306 ПОДДУБЬЕ ПСКОВСКАЯ ОБЛАСТЬ


In [15]:
data_name_group = all_data_in[['Группа продукции', 'Общее наименование продукции']]
data_name_group

,Группа продукции,Общее наименование продукции
0,Узлы пересечения противопожарных преград кабел...,Проходки кабельные модульные универсальные огн...
1,"Противопожарные окна, двери, двери шахт лифтов...",Ворота противопожарные металлические распашные...
2,"Клапаны противопожарные нормально открытые, кл...",Клапаны противопожарные марки КПВ.01(EI60) пря...
3,Вытяжные вентиляторы,Вентиляторы крышные радиальные дымоудаления с ...
4,Вытяжные вентиляторы,Вентиляторы радиальные дымоудаления типов ВР-8...
...,...,...
66748,Электрическое (электрооборудование) и неэлектр...,Блок пробоотборный обогреваемый
66749,Электрическое (электрооборудование) и неэлектр...,Система отбора проб природного газа
66750,Электрическое (электрооборудование) и неэлектр...,Фильтры
66751,Элементы оборудования (сборочные единицы) и ко...,Элементы трубопроводов


In [16]:
# prepare data:
prepared_data = data_name_group
for row in prepared_data.columns:
    prepared_data[row] = prepared_data[row].str.replace(r'\W', ' ')    
    prepared_data[row] = prepared_data[row].str.replace(r'\d', ' ')    
    prepared_data[row] = prepared_data[row].str.lower()
    prepared_data[row] = prepared_data[row].str.replace(r' +', ' ') 
    
prepared_data.head()

C:\Temp\ipykernel_63988\928657931.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  prepared_data[row] = prepared_data[row].str.replace(r'\W', ' ')
C:\Temp\ipykernel_63988\928657931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepared_data[row] = prepared_data[row].str.replace(r'\W', ' ')
C:\Temp\ipykernel_63988\928657931.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  prepared_data[row] = prepared_data[row].str.replace(r'\d', ' ')
C:\Temp\ipykernel_63988\928657931.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,Группа продукции,Общее наименование продукции
0,узлы пересечения противопожарных преград кабел...,проходки кабельные модульные универсальные огн...
1,противопожарные окна двери двери шахт лифтов с...,ворота противопожарные металлические распашные...
2,клапаны противопожарные нормально открытые кла...,клапаны противопожарные марки кпв ei прямоугол...
3,вытяжные вентиляторы,вентиляторы крышные радиальные дымоудаления с ...
4,вытяжные вентиляторы,вентиляторы радиальные дымоудаления типов вр д...


In [17]:
# Посмотреть, можеть добавить колонку с нумерацией

In [18]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = nltk.corpus.stopwords.words("russian")
stop_words.extend(['https', 'который', 'это', 'также', 'новый', 'являться', 'позволять', 'возможность', 
                   'наш', 'свой', 'ru', 'ещё', 'www', 'ru', 'com', 'пока', 'год'])

def lemmatize_cell(data_lemm, x, y):
    lemma = pymorphy2.MorphAnalyzer()
    cell = data_lemm.loc[y, data_lemm.columns[x]]
    print('Coords: ', x, y, '\n') #, datetime.datetime.now()
    text = ''
    if type(cell) == str and len(cell) > 0:
        text = " ".join([lemma.parse(word)[0].normal_form for word in cell.split(' ')])
        tokens = word_tokenize(text, language="russian")
#         tokens = lemma.parse(word)[0].normal_form 
        for token in tokens:
            if token in stop_words:
                tokens.remove(token)
        text = " ".join(tokens)
        
#     print(text, '\n')
        
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rexarrior\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [19]:
 prepared_data['Номер'] = range(1, len(prepared_data) + 1)
 prepared_data = prepared_data.reindex(columns=['Номер', 'Группа продукции', 'Общее наименование продукции'])
 prepared_data.tail(2)

C:\Temp\ipykernel_63988\3818430814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prepared_data['Номер'] = range(1, len(prepared_data) + 1)


,Номер,Группа продукции,Общее наименование продукции
66751,66752,элементы оборудования сборочные единицы и комп...,элементы трубопроводов
66752,66753,электрическое электрооборудование и неэлектрич...,станция газозаправочная автомобильная


In [20]:
%%time
data_lemm = prepared_data

data_lemm_out = pd.DataFrame()
data_lemm_out['Номер'] = data_lemm['Номер']
x_max = data_lemm.shape[1]
# x_max = 20
y_max = data_lemm.shape[0]
data_lemm_out = pd.DataFrame(columns=data_lemm.columns)

for x in range(x_max):
    for y in range(y_max):        
        cell = lemmatize_cell(data_lemm, x, y)
        data_lemm_out.loc[y, data_lemm_out.columns[x]] = cell
        
data_lemm_out

Coords:  0 0 

Coords:  0 1 

Coords:  0 2 

Coords:  0 3 

Coords:  0 4 

Coords:  0 5 

Coords:  0 6 

Coords:  0 7 

Coords:  0 8 

Coords:  0 9 

Coords:  0 10 

Coords:  0 11 

Coords:  0 12 

Coords:  0 13 

Coords:  0 14 

Coords:  0 15 

Coords:  0 16 

Coords:  0 17 

Coords:  0 18 

Coords:  0 19 

Coords:  0 20 

Coords:  0 21 

Coords:  0 22 

Coords:  0 23 

Coords:  0 24 

Coords:  0 25 

Coords:  0 26 

Coords:  0 27 

Coords:  0 28 

Coords:  0 29 

Coords:  0 30 

Coords:  0 31 

Coords:  0 32 

Coords:  0 33 

Coords:  0 34 

Coords:  0 35 

Coords:  0 36 

Coords:  0 37 

Coords:  0 38 

Coords:  0 39 

Coords:  0 40 

Coords:  0 41 

Coords:  0 42 

Coords:  0 43 

Coords:  0 44 

Coords:  0 45 

Coords:  0 46 

Coords:  0 47 

Coords:  0 48 

Coords:  0 49 

Coords:  0 50 

Coords:  0 51 

Coords:  0 52 

Coords:  0 53 

Coords:  0 54 

Coords:  0 55 

Coords:  0 56 

Coords:  0 57 

Coords:  0 58 

Coords:  0 59 

Coords:  0 60 

Coords:  0 61 

Coords:  0 62 

Co

In [ ]:
data_lemm_out.head()

: 

In [ ]:
data_lemm_out.to_excel(f'{dir_data_out}/data_lemm_morphy.xlsx', header=0)

: 

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.6, min_df=0.1, stop_words=stop_words)
data_vectors = vectorizer.fit_transform(data_lemm_out['Проект'])
data_vectors

: 

In [ ]:
# Determination of the required number of clusters
inertia = []
xy_min = 10
xy_max = 50
xy_step = 10
for k in range(xy_min,xy_max,xy_step):
    mbk  = MiniBatchKMeans(n_clusters=k,init='random', random_state=1).fit(data_vectors)
    inertia.append(np.sqrt(mbk.inertia_))
plt.plot(range(xy_min,xy_max,xy_step),inertia,marker='s')
plt.xlabel('K')
plt.ylabel('(C_k)')
plt.show()

# "Для того чтобы определить какое «K» нам нужно взять, нужно взглянуть на данный график и найти ту точку, 
# после которой инерция начнёт почти линейно уменьшаться." 

: 

In [ ]:
data_to_claster = data_lemm_out
data_to_claster.head()

: 

In [ ]:
# Clustering
n_clasters = 35
mbk  = MiniBatchKMeans(n_clusters=n_clasters,init='random').fit(data_vectors)
y_kmeansMBK = mbk.predict(data_vectors)
Num = [] 
Num = [pt for pt in y_kmeansMBK]
df2 = {'Кластер': Num}
dfMBK = pd.DataFrame(df2)
data_claster_out = pd.concat([data_to_claster,dfMBK], axis=1)
data_claster_out.to_excel(f'{dir_data_out}/Claster.xlsx', index=False)

: 

In [ ]:
data_all_out = pd.DataFrame()
data_all_out = all_data
data_all_out['Кластер'] = ''
for index, num in enumerate(data_claster_out['Номер']):
    data_all_out['Кластер'][num] = data_claster_out['Кластер'][index]
    
data_all_out['Кластер'].fillna('-', inplace=True)

data_all_out.to_excel(f'{dir_data_out}/all_data.xlsx', header=0)
data_all_out

: 

In [ ]:
data_claster_out

: 

: 